In [8]:
import os
import pandas as pd
from tqdm import tqdm
from glob import glob
from natsort import natsorted

In [9]:
# facemasked iamges directory
masked_img_folder = "./data/example_output/defaced_images"

# folder with 3D visualization GIFs
gif_folder = "./data/example_output/defaced_images_3d_visualization"

# unstripped original images for comparison with stripped images
orig_img_folder = "./data/example_input_images"

In [10]:
csv_filepath = "./data/example_output_manual_assessment.csv"
#df = pd.read_csv(os.path.join(csv_filepath))

In [11]:
img_folders = natsorted(glob(os.path.join(masked_img_folder, "*")))
gif_folders = natsorted(glob(os.path.join(gif_folder, "*")))
orig_folders = natsorted(glob(os.path.join(orig_img_folder, "*")))
len(img_folders), len(gif_folders), len(orig_folders)

(2, 2, 2)

In [12]:
orig_paths = natsorted(glob(os.path.join(orig_img_folder, "*", "*.nii.gz")))
img_paths = natsorted(glob(os.path.join(masked_img_folder, "*", "*_masked.nii.gz")))
gif_paths = natsorted(glob(os.path.join(gif_folder, "*", "*.gif")))
len(orig_paths), len(img_paths), len(gif_paths)

(2, 2, 2)

In [13]:
# build dataframe from gifs present in png_folder

df = pd.DataFrame(columns=["path_orig", "path_defaced", "path_gif", "deface_accepted"])

for orig_path, img_path, gif_path in zip(orig_paths, img_paths, gif_paths):
    row_list = [orig_path, img_path, gif_path, None]
    df.loc[len(df)] = row_list
df

,path_orig,path_defaced,path_gif,deface_accepted
0,./data/example_input_images/IXI002-Guys-0828-T...,./data/example_output/defaced_images/IXI002-Gu...,./data/example_output/defaced_images_3d_visual...,None
1,./data/example_input_images/IXI002-Guys-0828-T...,./data/example_output/defaced_images/IXI002-Gu...,./data/example_output/defaced_images_3d_visual...,None


In [14]:
for idx in range(0, len(df)):
    row = df.loc[idx]
    
    orig_path = row['path_orig']
    gif_path = row['path_gif']
    img_path = row['path_defaced']
    
    if os.path.isfile(gif_path):
        cmd = "mpv --speed=0.4 " + gif_path + " >/dev/null 2>&1"
        os.system(cmd)

        user_input = input()

        if user_input == "v":
            os.system(f"itksnap -g {img_path} -o {orig_path}")
            user_input = input()

        df.loc[idx, 'deface_accepted'] = user_input
        df.to_csv(csv_filepath, index=None)